(All programs were run on macOS, intel, 4 cores (8 logical cores), 8GB RAM)

EXERCISE 1: Profiling the Julia Code

In [ ]:
# Task 1.1 Calculate the Clock Granularity of different Python Timers (on your system)

# As part of the task measure and report the clock granularity (measured experimentally on your system) for 1) time.time(), 2) timeit and 3) time.time_ns() 
# (for the last one remember that time is outputted in ns!).

import numpy as np
import time
from timeit import default_timer as timer

def clock_granularity_time_time():
   M = 200
   timesfound = np.empty((M,))
   for i in range(M):
      t1 =  time.time()
      t2 = time.time()
      while (t2 - t1) < 1e-16: # if zero then we are below clock granularity, retake timing
          t2 = time.time()
      t1 = t2 # this is outside the loop
      timesfound[i] = t1 # record the time stamp
   minDelta = 1000000
   Delta = np.diff(timesfound) # it should be cast to int only when needed
   minDelta = Delta.min()
   return minDelta

def clock_granularity_timeit():
   M = 200
   timesfound = np.empty((M,))
   for i in range(M):
      t1 =  timer()
      t2 = timer()
      while (t2 - t1) < 1e-16: # if zero then we are below clock granularity, retake timing
          t2 = timer()
      t1 = t2 # this is outside the loop
      timesfound[i] = t1 # record the time stamp
   minDelta = 1000000
   Delta = np.diff(timesfound) # it should be cast to int only when needed
   minDelta = Delta.min()
   return minDelta


def clock_granularity_time_time_ns():
   M = 200
   timesfound = np.empty((M,))
   for i in range(M):
      t1 = time.time_ns()
      t2 = time.time_ns()
      while (t2 - t1) < 1e-16: # if zero then we are below clock granularity, retake timing
          t2 = time.time_ns()
      t1 = t2 # this is outside the loop
      timesfound[i] = t1 # record the time stamp
   minDelta = 1000000
   Delta = np.diff(timesfound) # it should be cast to int only when needed
   minDelta = Delta.min()
   return minDelta

granularity_time_time = clock_granularity_time_time()
granularity_timeit = clock_granularity_timeit()
granularity_time_time_ns = clock_granularity_time_time_ns() / 1_000_000_000
print("Results:")
print("time.time(): ", granularity_time_time)
print("timeit: ", granularity_timeit)
print("time.time_ns(): ", granularity_time_time_ns)

In [ ]:
# Task 1.2 Timing the Julia set code functions.

"""
The goal is to time the calc_pure_python and calculate_z_serial_purepython. 
We ask you to develop a decorator to wrap the functions to be profiled for this task. 
Use the decorator to add timer functionality for time measurements with the best timer you found in the previous task. 

As part of the task:
    - Develop the timer decorator 
    - Provide timing information for the two functions. Report averages and standard deviation. How does the standard deviation compare to the clock frequency?
"""

# ------------------------- JULIA CODE ----------------------------------------
"""Julia set generator without optional PIL-based image drawing"""
import time
import numpy as np
from functools import wraps
from timeit import default_timer as timer

# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193

# decorator to time
def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = timer()
        result = fn(*args, **kwargs)
        t2 = timer()
        timing = t2-t1
        if fn.__name__ == "calc_pure_python":
            calc_pure_python_timings.append(timing)
        else:
            calc_z_serial_timings.append(timing)
        return result
    return measure_time

@timefn
def calc_pure_python(desired_width, max_iterations):
    """Create a list of complex coordinates (zs) and complex parameters (cs),
    build Julia set"""
    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list of coordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our
    # function

    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    output = calculate_z_serial_purepython(max_iterations, zs, cs)

    # This sum is expected for a 1000^2 grid with 300 iterations
    # It ensures that our code evolves exactly as we'd intended
    # assert sum(output) == 33219980

@timefn
def calculate_z_serial_purepython(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

if __name__ == "__main__":

    # Calculate the Julia set using a pure Python solution with
    # reasonable defaults for a laptop
    calc_pure_python_timings = []
    calc_z_serial_timings = []
    
    for i in range(100):
        calc_pure_python(desired_width=1000, max_iterations=300)
    
    calc_pure_python_timings = np.array(calc_pure_python_timings)
    calc_z_serial_timings = np.array(calc_z_serial_timings)

    calc_pure_python_mean = np.mean(calc_pure_python_timings)
    calc_z_serial_mean = np.mean(calc_z_serial_timings)

    calc_pure_python_std = np.std(calc_pure_python_timings)
    calc_z_serial_std = np.std(calc_z_serial_timings)

    print("calc pure: ", calc_pure_python_timings)
    print("calc_serial: ", calc_z_serial_timings)

    print("calc pure mean: ", calc_pure_python_mean)
    print("calc_serial mean: ", calc_z_serial_mean)

    print("calc pure std: ", calc_pure_python_std)
    print("calc_serial std: ", calc_z_serial_std)

# -------------------------------------------------------------------------------

In [ ]:
# Task 1.3 Profile the Julia set code with cProfile and line_profiler the computation

"""
Use the cProfile and line_profiler to profile the computation in JuliaSet code.

As part of the task:

     -  Report the results of cProfile and line_profiler (for the two functions)

     - Use SnakeViz to visualize the profiling information from cProfile

     - Measure the overhead added by using cProfile and line_profiler. For this, you can time the code with and without  the profilers 
"""

#---------- Profiling using cprofile ------------
#To print cProfile data in the terminal:
!python3 -m cProfile -s cumulative JuliaSet.py

In [ ]:
#To have cProfile output performance statistics to a file:
!python3 -m cProfile -o profile.stats JuliaSet.py

In [ ]:
#Then, we can either use the "readStats.py" file to sort and display the data in terminal:
!pip3 install pstats
!python3 readStats.py

In [ ]:

#Or we can use snakeviz to visualise the data, which displays the data on a browser page
!pip3 install snakeviz
!python3 -m snakeviz profile.stats --server

In [ ]:
#---------- Profiling using line profiler ------------
# By adding "@profile" in the line above the calc_pure_python and calculate_z_serial_purepython functions, we get data on the duration of each line within these functions
# We print this data to terminal by using the following two commands:
# (JuliaSetWithProfile.py is identical to JuliaSet.py, but with the profile wrapper added to the functions)
!python3 -m kernprof -l JuliaSetWithProfile.py
!python3 -m line_profiler JuliaSetWithProfile.py.lprof

In [10]:
# #---------- Measuring the overhead of using these two tools: ------------
# #The file JuliaSetTiming.py is identical to JuliaSet.py, but it also measures the time it takes to execute the calc_pure_python function
# #Without any tools:
print("Without timing, calc_pure_python takes this long in seconds:")
!python3 JuliaSetTiming.py

#Using c profile tool:
print("Using the c profile tool, calc_pure_python takes this long in seconds:")
!python3 -m cProfile -o profile.stats JuliaSetTiming.py

#Using line profiling tool: (we use JuliaSetTimingLineProfile because it also incudes the @profile wraps needed)
print("Using the line profiling tool, calc_pure_python takes this long in seconds:")
!python3 -m kernprof -l JuliaSetTimingLineProfile.py

function took this long: 
0.014415491983527318
function took this long: 
0.03183580600307323
function took this long: 
0.19322470400948077
Wrote profile results to JuliaSetTimingLineProfile.py.lprof
Inspect results with:
python3 -m line_profiler -rmt "JuliaSetTimingLineProfile.py.lprof"


We can see from the results that the timeit timer has the smallest granularity, and is therefore the most accurate. However, all of the results are in the same order of magnitude, and so are very similar.

In [1]:
#Task 1.4 Memory-profile the Juliaset code. Use the memory_profiler and mprof to profile the computation in JuliaSet code
"""
Use memory_profiler to profile the memory usage for the two functions and use mprof to collect and visualize the profiling information.

As part of the task:

    - Report the memory profiling results from memory_profiler and mprof (including the plot)

    - Measure the overhead of memory_profiler and mprof.
"""
# !pip3 install memory_profiler
# !pip3 install psutil
#Add profile decorator before each function you want to analyse
!python3 -m kernprof -l JuliaSet.py
!python3 -m line_profiler -rmt JuliaSet.py.lprof

Length of x: 100
Total elements: 10000
calculate_z_serial_purepython took 0.010776042938232422 seconds
Wrote profile results to JuliaSet.py.lprof
Inspect results with:
python3 -m line_profiler -rmt "JuliaSet.py.lprof"
Timer unit: 1e-06 s



EXERCISE 2: PROFILING DIFFUSION PROCESS CODE

In [ ]:
#---------- 2.1 Profiling using cProfile ------------
#To print cProfile data in the terminal:
!python3 -m cProfile -s diffusion.py

In [ ]:
#To have cProfile output performance statistics to a file:
!python3 -m cProfile -o profile.stats diffusion.py

In [ ]:
#Then, we can either use the "readStats.py" file to sort and display the data in terminal:
!pip3 install pstats
!python3 readStats.py

In [ ]:
#Or we can use snakeviz to visualise the data, which displays the data on a browser page
!pip3 install snakeviz
!python3 -m snakeviz profile.stats --server

In [ ]:
#---------- Profiling using line profiler ------------
!pip3 install line_profiler
!python3 -m kernprof -l diffusionWithProfile.py

In [ ]:
#--------- 2.2: Memory profiler ------------
!pip install memory_profiler
!pip install psutil

In [ ]:
# Memory profiler. Large overhead, so a smaller problem size was used
!python -m memory_profiler diffusion_smaller.py

In [ ]:
# This command saves the output to a file named mprofile_20240130... or similar
!python -m mprof run diffusion.py

In [ ]:
# Change the name of the file to be identical with the one produced by the previous command
!python -m mprof plot mprofile_20240127154859